In [2]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping, MultiPoint
import scipy.interpolate
import geopy
import geopy.distance
import time
import folium
import gmaps
import networkx as nx
from shapely.ops import nearest_points
import itertools

In [3]:
distance_G_busy = nx.read_gpickle('D:/Datasets/MIMOS/kl_selangor_traffic_flow_26082020_0700_0800_distance.gpickle')
roadCondition_G_busy = nx.read_gpickle('D:/Datasets/MIMOS/kl_selangor_traffic_flow_26082020_0700_0800_roadCondition.gpickle')
distance_G_off = nx.read_gpickle('D:/Datasets/MIMOS/kl_selangor_traffic_flow_26082020_0000_0100_distance.gpickle')
roadCondition_G_off = nx.read_gpickle('D:/Datasets/MIMOS/kl_selangor_traffic_flow_26082020_0000_0100_roadCondition.gpickle')

In [4]:
print(len(distance_G_busy.nodes))
print(len(roadCondition_G_busy.nodes))
print(len(distance_G_off.nodes))
print(len(roadCondition_G_off.nodes))

144316
144316
144316
144316


In [6]:
# Sample points
monash = Point(3.064932, 101.601016)
mimos = Point(3.046843, 101.696183)
home = Point(3.027020, 101.570559)

sunway = Point(3.071410, 101.607970)
klsentral = Point(3.133816, 101.684744)
klcc = Point(3.15784,101.712135)

### Get nearest point from network for both source and destination and find the shortest path

In [8]:
graph_nodes = []
for element in distance_G_busy.nodes:
    #print(element)
    graph_nodes.append(Point(element))

# To change the Point to graph node readable format
def get_node(x):
    x = x.replace('(','')
    x = x.replace(')','')
    x = x.split(' ')
    
    return float(x[1]), float(x[2])

# To get the node closest to the input Point
def get_graph_node(x):
    return [o.wkt for o in nearest_points(x, MultiPoint(graph_nodes))][1]

In [9]:
str(get_node(get_graph_node(mimos))) + str(1)
str(get_node(get_graph_node(klcc))) + str(1)

'(3.159, 101.71258)1'

# Building the multi layer graph

In [10]:
stops = [mimos,home, sunway, klsentral]
markers = [monash, mimos, home,sunway,klsentral, klcc]
layer_counter = 0
multi_layer_graph = nx.Graph()

start_layer = nx.relabel_nodes(distance_G_busy, lambda x: str(x) + "_" +str(layer_counter))
multi_layer_graph.add_nodes_from(start_layer)
multi_layer_graph.add_edges_from(start_layer.edges())
layer_counter += 1

for i in range(len(stops)):
    print(i, stops)
    mid_layer = nx.relabel_nodes(distance_G_busy, lambda x: str(x) + "_" + str(layer_counter)) 
    multi_layer_graph.add_nodes_from(mid_layer)
    multi_layer_graph.add_edges_from(mid_layer.edges())
    layer_counter += 1
    
    if i == 0:
        connecting_point_1 = str(get_node(get_graph_node(markers[i]))) + "_" + str(i)
        connecting_point_2 = str(get_node(get_graph_node(markers[i]))) + "_" + str(i+1)
    else:
        connecting_point_1 = str(get_node(get_graph_node(stops[i-1]))) + "_" + str(i)
        connecting_point_2 = str(get_node(get_graph_node(stops[i-1]))) + "_" + str(i+1)
    
    print(connecting_point_1, connecting_point_2)
    multi_layer_graph.add_edge(connecting_point_1, connecting_point_2, weight =0)
    

end_layer = nx.relabel_nodes(distance_G_busy, lambda x: str(x) + "_" + str(layer_counter))
multi_layer_graph.add_nodes_from(end_layer)
multi_layer_graph.add_edges_from(end_layer.edges())
connecting_point_1 = str(get_node(get_graph_node(stops[-1]))) + "_" + str(layer_counter - 1)
connecting_point_2 = str(get_node(get_graph_node(stops[-1]))) + "_" + str(layer_counter)
print(connecting_point_1, connecting_point_2)
multi_layer_graph.add_edge(connecting_point_1, connecting_point_2, weight =0)

0 [<shapely.geometry.point.Point object at 0x0000024471D085F8>, <shapely.geometry.point.Point object at 0x000002440AECC320>, <shapely.geometry.point.Point object at 0x000002440D3985F8>, <shapely.geometry.point.Point object at 0x0000024471C72E48>]
(3.06528, 101.59911)_0 (3.06528, 101.59911)_1
1 [<shapely.geometry.point.Point object at 0x0000024471D085F8>, <shapely.geometry.point.Point object at 0x000002440AECC320>, <shapely.geometry.point.Point object at 0x000002440D3985F8>, <shapely.geometry.point.Point object at 0x0000024471C72E48>]
(3.04941, 101.69611)_1 (3.04941, 101.69611)_2
2 [<shapely.geometry.point.Point object at 0x0000024471D085F8>, <shapely.geometry.point.Point object at 0x000002440AECC320>, <shapely.geometry.point.Point object at 0x000002440D3985F8>, <shapely.geometry.point.Point object at 0x0000024471C72E48>]
(3.02693, 101.57018)_2 (3.02693, 101.57018)_3
3 [<shapely.geometry.point.Point object at 0x0000024471D085F8>, <shapely.geometry.point.Point object at 0x000002440AECC32

In [95]:
test_route = nx.dijkstra_path(multi_layer_graph, str(get_node(get_graph_node(monash))) + "_" + str(0), str(get_node(get_graph_node(klcc))) + "_" + str(len(stops) + 1))
final_route = []
for element in test_route:
    #print(element.split('_')[0])
    coords = element.split('_')[0]
    coords = coords.replace('(','')
    coords = coords.replace(')','')
    #print(coords)
    lat = float(coords.split(',')[0])
    lon = float(coords.split(',')[1])
    #print(type((lat,lon)))
    #print((lat,lon))
    final_route.append((lat,lon))
kl_selangor_map = folium.Map(location=[3.027020, 101.570559], zoom_start= 10)
folium.PolyLine(final_route, color="blue", weight=10, opacity=0.5).add_to(kl_selangor_map)
folium.Marker([3.064932, 101.601016], popup='Monash').add_to(kl_selangor_map)
folium.Marker([3.046843, 101.696183], popup='MIMOS').add_to(kl_selangor_map)
folium.Marker([3.027020, 101.570559], popup='Home').add_to(kl_selangor_map)
folium.Marker([3.15784,101.712135], popup='KLCC').add_to(kl_selangor_map)


In [96]:
kl_selangor_map

In [74]:
# Sample points
monash = Point(3.064932, 101.601016)
mimos = Point(3.046843, 101.696183)
home = Point(3.027020, 101.570559)
klcc = Point(3.15784,101.712135)

sample_points = []
sample_points.append(monash)
sample_points.append(mimos)
sample_points.append(home)
sample_points.append(klcc)

kl_selangor_map = folium.Map(location=[3.027020, 101.570559], zoom_start= 10)

route_arr = []
for comb in itertools.combinations(sample_points, 2):
    print(comb)
    print(comb[0], comb[1])
    route = nx.dijkstra_path(distance_G_busy, get_node(get_graph_node(comb[0])), get_node(get_graph_node(comb[1])))
    route2 = nx.dijkstra_path(roadCondition_G_busy, get_node(get_graph_node(comb[0])), get_node(get_graph_node(comb[1])))
    route3 = nx.dijkstra_path(distance_G_off, get_node(get_graph_node(comb[0])), get_node(get_graph_node(comb[1])))
    route4 = nx.dijkstra_path(roadCondition_G_off, get_node(get_graph_node(comb[0])), get_node(get_graph_node(comb[1])))
    
    temp_arr = []
    temp_arr.append(get_node(get_graph_node(comb[0])))
    temp_arr.append(get_node(get_graph_node(comb[1])))
    temp_arr.append(route)
    temp_arr.append('distance')
    temp_arr.append('busy')
    
    temp_arr2 = []
    temp_arr2.append(get_node(get_graph_node(comb[0])))
    temp_arr2.append(get_node(get_graph_node(comb[1])))
    temp_arr2.append(route)
    temp_arr2.append('roadCondition')
    temp_arr2.append('busy')
    
    temp_arr3 = []
    temp_arr3.append(get_node(get_graph_node(comb[0])))
    temp_arr3.append(get_node(get_graph_node(comb[1])))
    temp_arr3.append(route)
    temp_arr3.append('distance')
    temp_arr3.append('off')
    
    temp_arr4 = []
    temp_arr4.append(get_node(get_graph_node(comb[0])))
    temp_arr4.append(get_node(get_graph_node(comb[1])))
    temp_arr4.append(route)
    temp_arr4.append('roadCondition')
    temp_arr4.append('off')
    
    route_arr.append(temp_arr)
    route_arr.append(temp_arr2)
    route_arr.append(temp_arr3)
    route_arr.append(temp_arr4)
    
    
    folium.PolyLine(route1, color="blue", weight=10, opacity=0.5).add_to(kl_selangor_map)
    folium.PolyLine(route2, color="red", weight=5, opacity=0.5).add_to(kl_selangor_map)
    folium.PolyLine(route3, color="orange", weight=5, opacity=0.5).add_to(kl_selangor_map)
    folium.PolyLine(route4, color="green", weight=5, opacity=0.5).add_to(kl_selangor_map)
    

(<shapely.geometry.point.Point object at 0x0000016183416630>, <shapely.geometry.point.Point object at 0x00000161834165F8>)
POINT (3.064932 101.601016) POINT (3.046843 101.696183)
(<shapely.geometry.point.Point object at 0x0000016183416630>, <shapely.geometry.point.Point object at 0x00000161800316A0>)
POINT (3.064932 101.601016) POINT (3.02702 101.570559)
(<shapely.geometry.point.Point object at 0x0000016183416630>, <shapely.geometry.point.Point object at 0x00000161834166A0>)
POINT (3.064932 101.601016) POINT (3.15784 101.712135)
(<shapely.geometry.point.Point object at 0x00000161834165F8>, <shapely.geometry.point.Point object at 0x00000161800316A0>)
POINT (3.046843 101.696183) POINT (3.02702 101.570559)
(<shapely.geometry.point.Point object at 0x00000161834165F8>, <shapely.geometry.point.Point object at 0x00000161834166A0>)
POINT (3.046843 101.696183) POINT (3.15784 101.712135)
(<shapely.geometry.point.Point object at 0x00000161800316A0>, <shapely.geometry.point.Point object at 0x00000

In [75]:
route_df = pd.DataFrame(route_arr)
route_df.columns = ['source', 'destination', 'route', 'network_type', 'time_period']
route_df.head()

,source,destination,route,network_type,time_period
0,"(3.06528, 101.59911)","(3.04941, 101.69611)","[(3.06528, 101.59911), (3.06528, 101.59898), (...",distance,busy
1,"(3.06528, 101.59911)","(3.04941, 101.69611)","[(3.06528, 101.59911), (3.06528, 101.59898), (...",roadCondition,busy
2,"(3.06528, 101.59911)","(3.04941, 101.69611)","[(3.06528, 101.59911), (3.06528, 101.59898), (...",distance,off
3,"(3.06528, 101.59911)","(3.04941, 101.69611)","[(3.06528, 101.59911), (3.06528, 101.59898), (...",roadCondition,off
4,"(3.06528, 101.59911)","(3.02693, 101.57018)","[(3.06528, 101.59911), (3.06528, 101.59898), (...",distance,busy


In [76]:
kl_selangor_map

In [77]:
route_df.to_csv('D:\Datasets\MIMOS\sample_routes.csv',index=False)

In [61]:
## Change the point here
source = Point(3.064932, 101.601016)
destination = Point(3.046843, 101.696183)
#print(home)
graph_nodes = []
for element in distance_G_busy.nodes:
    #print(element)
    graph_nodes.append(Point(element))

source_G = [o.wkt for o in nearest_points(source, MultiPoint(graph_nodes))][1]
destination_G = [o.wkt for o in nearest_points(destination, MultiPoint(graph_nodes))][1]
print([o.wkt for o in nearest_points(source, MultiPoint(graph_nodes))])
print([o.wkt for o in nearest_points(destination, MultiPoint(graph_nodes))])
print(source_G)
print(destination_G)

#nearest = nearest_points(home, list(kl_selangor_graph_distance.nodes))

['POINT (3.064932 101.601016)', 'POINT (3.06528 101.59911)']
['POINT (3.046843 101.696183)', 'POINT (3.04941 101.69611)']
POINT (3.06528 101.59911)
POINT (3.04941 101.69611)


In [62]:
route1 = nx.dijkstra_path(distance_G_busy, get_node(source_G), get_node(destination_G))

In [80]:
for element in test_route:
    print(element.split('_')[0])
    coords = element.split('_')[0]
    coords = coords.replace('(','')
    coords = coords.replace(')','')
    lat = element.split(',')[0]
    lon = element.split(',')[1]
    print(type((lat,lon)))
    final_route.append((lat,lon))

(3.06528, 101.59911)
<class 'tuple'>
(3.06528, 101.59911)
<class 'tuple'>
(3.06528, 101.59898)
<class 'tuple'>
(3.06528, 101.59887)
<class 'tuple'>
(3.06528, 101.59876)
<class 'tuple'>
(3.06524, 101.59884)
<class 'tuple'>
(3.06518, 101.59887)
<class 'tuple'>
(3.06433, 101.59887)
<class 'tuple'>
(3.06403, 101.59885)
<class 'tuple'>
(3.06376, 101.5988)
<class 'tuple'>
(3.06294, 101.59878)
<class 'tuple'>
(3.06277, 101.59876)
<class 'tuple'>
(3.0625, 101.5987)
<class 'tuple'>
(3.06232, 101.59863)
<class 'tuple'>
(3.06222, 101.5986)
<class 'tuple'>
(3.06206, 101.59857)
<class 'tuple'>
(3.06199, 101.59868)
<class 'tuple'>
(3.06186, 101.59889)
<class 'tuple'>
(3.06176, 101.59903)
<class 'tuple'>
(3.06124, 101.59966)
<class 'tuple'>
(3.061, 101.59994)
<class 'tuple'>
(3.06082, 101.60019)
<class 'tuple'>
(3.0606, 101.60051)
<class 'tuple'>
(3.06054, 101.60061)
<class 'tuple'>
(3.06041, 101.60087)
<class 'tuple'>
(3.06029, 101.60114)
<class 'tuple'>
(3.06017, 101.60142)
<class 'tuple'>
(3.0601,

In [87]:
for element in test_route:
    print(element.split('_')[0])
    coords = element.split('_')[0]
    coords = coords.replace('(','')
    coords = coords.replace(')','')
    print(coords)
    lat = float(coords.split(',')[0])
    lon = float(coords.split(',')[1])
    print(type((lat,lon)))
    print((lat,lon))
    final_route.append((lat,lon))

(3.06528, 101.59911)
3.06528, 101.59911
<class 'tuple'>
('3.06528', ' 101.59911')
(3.06528, 101.59911)
3.06528, 101.59911
<class 'tuple'>
('3.06528', ' 101.59911')
(3.06528, 101.59898)
3.06528, 101.59898
<class 'tuple'>
('3.06528', ' 101.59898')
(3.06528, 101.59887)
3.06528, 101.59887
<class 'tuple'>
('3.06528', ' 101.59887')
(3.06528, 101.59876)
3.06528, 101.59876
<class 'tuple'>
('3.06528', ' 101.59876')
(3.06524, 101.59884)
3.06524, 101.59884
<class 'tuple'>
('3.06524', ' 101.59884')
(3.06518, 101.59887)
3.06518, 101.59887
<class 'tuple'>
('3.06518', ' 101.59887')
(3.06433, 101.59887)
3.06433, 101.59887
<class 'tuple'>
('3.06433', ' 101.59887')
(3.06403, 101.59885)
3.06403, 101.59885
<class 'tuple'>
('3.06403', ' 101.59885')
(3.06376, 101.5988)
3.06376, 101.5988
<class 'tuple'>
('3.06376', ' 101.5988')
(3.06294, 101.59878)
3.06294, 101.59878
<class 'tuple'>
('3.06294', ' 101.59878')
(3.06277, 101.59876)
3.06277, 101.59876
<class 'tuple'>
('3.06277', ' 101.59876')
(3.0625, 101.5987)


(3.03118, 101.57456)
3.03118, 101.57456
<class 'tuple'>
('3.03118', ' 101.57456')
(3.03171, 101.57443)
3.03171, 101.57443
<class 'tuple'>
('3.03171', ' 101.57443')
(3.03272, 101.57413)
3.03272, 101.57413
<class 'tuple'>
('3.03272', ' 101.57413')
(3.03361, 101.57385)
3.03361, 101.57385
<class 'tuple'>
('3.03361', ' 101.57385')
(3.03499, 101.57338)
3.03499, 101.57338
<class 'tuple'>
('3.03499', ' 101.57338')
(3.0363, 101.57294)
3.0363, 101.57294
<class 'tuple'>
('3.0363', ' 101.57294')
(3.03751, 101.57251)
3.03751, 101.57251
<class 'tuple'>
('3.03751', ' 101.57251')
(3.03823, 101.57225)
3.03823, 101.57225
<class 'tuple'>
('3.03823', ' 101.57225')
(3.03952, 101.57172)
3.03952, 101.57172
<class 'tuple'>
('3.03952', ' 101.57172')
(3.04034, 101.57137)
3.04034, 101.57137
<class 'tuple'>
('3.04034', ' 101.57137')
(3.04182, 101.57066)
3.04182, 101.57066
<class 'tuple'>
('3.04182', ' 101.57066')
(3.04245, 101.57037)
3.04245, 101.57037
<class 'tuple'>
('3.04245', ' 101.57037')
(3.04408, 101.5696)

In [94]:
test_route

['(3.06528, 101.59911)_0',
 '(3.06528, 101.59911)_1',
 '(3.06528, 101.59898)_1',
 '(3.06528, 101.59887)_1',
 '(3.06528, 101.59876)_1',
 '(3.06524, 101.59884)_1',
 '(3.06518, 101.59887)_1',
 '(3.06433, 101.59887)_1',
 '(3.06403, 101.59885)_1',
 '(3.06376, 101.5988)_1',
 '(3.06294, 101.59878)_1',
 '(3.06277, 101.59876)_1',
 '(3.0625, 101.5987)_1',
 '(3.06232, 101.59863)_1',
 '(3.06222, 101.5986)_1',
 '(3.06206, 101.59857)_1',
 '(3.06199, 101.59868)_1',
 '(3.06186, 101.59889)_1',
 '(3.06176, 101.59903)_1',
 '(3.06124, 101.59966)_1',
 '(3.061, 101.59994)_1',
 '(3.06082, 101.60019)_1',
 '(3.0606, 101.60051)_1',
 '(3.06054, 101.60061)_1',
 '(3.06041, 101.60087)_1',
 '(3.06029, 101.60114)_1',
 '(3.06017, 101.60142)_1',
 '(3.0601, 101.60165)_1',
 '(3.06004, 101.60184)_1',
 '(3.0599, 101.60236)_1',
 '(3.05972, 101.60316)_1',
 '(3.0595, 101.60406)_1',
 '(3.05926, 101.60503)_1',
 '(3.05913, 101.6055)_1',
 '(3.05905, 101.60579)_1',
 '(3.05895, 101.60614)_1',
 '(3.05889, 101.60634)_1',
 '(3.05877, 